# JEM092 Asset Pricing - Homework 1

<span style='background: yellow'> **Group : 20107894**</span>

Team Member:

Yanqi Guan : 20107894@fsv.cuni.cz  
Lin Zhang : 15845542@fsv.cuni.cz  
Jiayi Zeng : 46566215@fsv.cuni.cz


### Task 1 - Data download (8 pts)
In the previous step, you were assigned firms for which you will get the data. You will download 
* daily adjusted close price and volume data from www.finance.yahoo.com
* Market Capitalization and Book Value per Share data from www.macrotrends.net
* Sample period: 01.01.2007 - 28.02.2022 

### (1). Load stock symbols from file

In [ ]:
# Setup environment
Sys.setenv(LANG = "en")
#options(warn = -1)  # suppressing warnings

if (!require(quantmod)) install.packages('quantmod')
if (!require(httr)) install.packages('httr')
if (!require(rvest)) install.packages('rvest')
if (!require(jsonlite)) install.packages('jsonlite')
if (!require(treasuryTR)) install.packages('treasuryTR')


library(quantmod)
library(httr)
library(rvest)
library(jsonlite)
library(treasuryTR)

In [ ]:
# read data from file
smbs <- read.csv('20107894_rand_download.csv',colClasses = "character")
head(smbs)
symbols <- smbs[[2]]
symbols

### (2). Download Trading Data

In [ ]:
# Download closing price and volume data into list named OHLCV
OHLCV <- lapply(symbols, function(y)
{
    try(getSymbols(y, auto.assign = FALSE,from = as.Date('2007-01-01'), to = '2022-02-28'),silent=TRUE)
})

#Assign symbols name
names(OHLCV) <- symbols

print("Closing Prices are downloaded successfully !")

In [ ]:
# Save data into file if necessary
# save(OHLCV, file = "1-price.RData")
# load("1-price.RData")

# check the data, symbol names and counts
length(OHLCV)
attributes(OHLCV)$names

# check data date range
head(OHLCV[[1]])
tail(OHLCV[[1]])

In [ ]:
# Check all symbols get data
lapply(OHLCV, head)



### (3). Download Book Value Data

In [ ]:
# Setup global parameters
m_website <- "https://www.macrotrends.net/stocks/charts/"
m_pb <- "price-book"
m_website
m_pb

In [ ]:
#  Searching for company names, create webpage link from symbols
getLink <- function(symb){    
    r1 <- GET(paste0(m_website, symb))
    return (paste0(r1[[1]] , m_pb))
}

# Call the function and output updated links
links <- lapply(symbols, getLink)
links

In [ ]:
# Download PB data from website, and put data into xts list named "book_value"
# This process may run a while ...

# Create download data function
getPB <- function(symbol_in){   
    
    # Get the links
    r1 <- GET(paste0(m_website, symbol_in))
    link_in <- (paste0(r1[[1]] , m_pb))       
    
    # Read HTML data from webpage
    t_table <- read_html(link_in) %>% 
        html_table(header = TRUE, fill = TRUE ) %>% 
            .[[1]]    
    
    # Rename table columns
    colnames(t_table) <- c("date", "Stock Price", "Book Value per Share",
                            "Price/Book Ratio")    
    # Clean data format
    t_table <- t_table[-1,] # Remove first row column names
    t_table[[1]] <- as.Date(t_table[[1]]) # Format date
    t_table[[2]] <- as.numeric(t_table[[2]]) # Format Stock Price
    t_table[[3]] <- as.numeric(gsub("[$,]","",t_table[[3]])) # Format Book Value
    t_table[[4]] <- as.numeric(t_table[[4]]) # Format PB
    
    # Put html_table into xts format
    t_xts <- tibble_to_xts(t_table)

    # Put xts object into global list
    book_value[[symbol_in]] <<- t_xts 
    
    return (paste0(symbol_in , ":  Book Value data is downloaded successfully !"))
}

# Create a xts list for book value data
book_value <- list()

# call getPB function for 20 symbols
lapply(symbols, getPB)


In [ ]:
# Save data into file if necessary
# save(book_value, file = "2-PB.RData")
# load("2-PB.RData")

# check the data, symbol names and counts
length(book_value)
attributes(book_value)$names

# check data date range
head(book_value[[1]])
tail(book_value[[1]])

In [ ]:
# check all symbols get data
lapply(book_value, head)

### (4). Download Market Cap Data

In [ ]:
# Setup global parameters
mc_website <- "https://www.macrotrends.net/assets/php/market_cap.php?t="
mc_website

In [ ]:
# Download Market Cap data from website, and put data into xts list named "MktCap"
# This process may run a while ...

# Create download data function
getMC <- function(symbol_in){   
    
    # Get the links     
    link_in <- paste0(mc_website, symbol_in)
    
    print(link_in)
    # Read HTML data from webpage
    m_node <- read_html(link_in) %>% 
        html_node('body') %>%
            html_children() %>%  
                html_text() 
    
    # Parse html data
    s1 <- strsplit(m_node, split = "]")
    s2 <- strsplit(s1[[2]][1], split = "=")
    s3 <- paste0(s2[[1]][2], "]")

    # call jsonlite library, get data frame
    mc_j <- fromJSON(s3)    
    
    # Convert data frame to time series
    mc_xts <- xts(mc_j[,2], as.Date(mc_j$date))        

    # Put xts object into global list
    MktCap[[symbol_in]] <<- mc_xts 
    
    return (paste0(symbol_in , ":  Market Cap data is downloaded successfully !"))
}

# Create a xts list for book value data
MktCap <- list()

# call getPB function for 20 symbols
lapply(symbols, getMC)


In [ ]:
# Save data into file if necessary
# save(MktCap, file = "3-MC.RData")
# load("3-MC.RData")

# check the data, symbol names and counts
length(MktCap)
attributes(MktCap)$names

# check data date range
head(MktCap[[1]])
tail(MktCap[[1]])

In [ ]:
# check all symbols get data
lapply(MktCap, head)